In [1]:
!pip install scikit-learn
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection, datasets, linear_model, metrics

### Ансамбль методов для классификации

### Стекинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

#### Обработка датасета

# Классификация

In [2]:
credit_df = pd.read_csv('../data/Credit_risk_preprocessed.csv')


In [3]:
credit_df["default"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(x, y, test_size = 0.3, shuffle=False)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_cls)

X_train_cls = scaler.transform(X_train_cls)
X_test_cls = scaler.transform(X_test_cls)

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

In [4]:
estimators = [('lr', LogisticRegression()), ('dt', DecisionTreeClassifier())]
modelClf1 = StackingClassifier(estimators=estimators)

modelClf1.fit(X_train_cls, y_train_cls)
print(modelClf1.score(X_test_cls, y_test_cls))

0.9666666666666667


#### Подбор гиперпараметров для базового алгоритма дерева решений

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':[1, 2, 3, 4, 5], 'min_samples_leaf': range (1, 8) ,'min_samples_split': range (2,10,2)}
dtc = tree.DecisionTreeClassifier()
grid1_ = GridSearchCV(dtc, parameters)

In [6]:
grid1_.fit(X_train_cls, y_train_cls)
print(grid1_.best_params_)
print(grid1_.best_estimator_)

{'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 2}
DecisionTreeClassifier(max_depth=5, min_samples_leaf=3)


In [7]:
tr1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 1, min_samples_split = 2)
tr1 = tr1.fit(X_train_cls, y_train_cls)
y_pred1 = tr1.predict(X_test_cls)
print(tr1.score(X_test_cls, y_test_cls))

0.9416666666666667


#### Подбор гиперпараметров для базового алгоритма логистической регрессии

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'penalty': ('l1', 'l2', 'elasticnet', 'none'),'solver' : ['liblinear', 'sag', 'saga'], 'C': np.logspace(-4, 4, 20) }
lr = LogisticRegression()
grid2_ = GridSearchCV(lr, parameters)
grid2_.fit(X_train_cls, y_train_cls)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ['liblinear', 'sag', 'saga']})

In [9]:
print(grid2_.best_params_)
print(grid2_.best_estimator_)

{'C': 0.23357214690901212, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=0.23357214690901212, solver='liblinear')


In [10]:
lr1 = LogisticRegression(C=1.623776739188721, penalty='l1', solver='liblinear')
lr1.fit(X_train_cls, y_train_cls)
y_pred = lr1.predict(X_test_cls)
print(lr1.score(X_test_cls, y_test_cls))

0.945


## Бэггинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

### Бэггинг без подбора гиперпараметров для него и базовых алгоритмов

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf3 = BaggingClassifier(base_estimator=LogisticRegression())

modelClf3.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf3.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.94


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf4 = BaggingClassifier(base_estimator=DecisionTreeClassifier())

modelClf4.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Дерево решений":',modelClf4.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Дерево решений": 0.9866666666666667


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf5 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'))

modelClf5.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров":',modelClf5.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров": 0.9483333333333334


#### Подбор параметров для бэггинга

In [15]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgc1 = BaggingClassifier(base_estimator=LogisticRegression())
grid4_ = GridSearchCV(bgc1, parameters)

In [16]:
grid4_.fit(X_train_cls, y_train_cls)
print(grid4_.best_params_)
print(grid4_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 9}
BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=9)


In [17]:
bgc2 = BaggingClassifier(base_estimator=LogisticRegression(), max_samples=1.0, n_estimators=2)
bgc2.fit(X_train_cls, y_train_cls)
print(bgc2.score(X_test_cls, y_test_cls))

0.9383333333333334


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf6 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'),max_samples=1.0, n_estimators=5)

modelClf6.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf6.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.9483333333333334


In [19]:
bgc3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth = 1, min_samples_leaf = 1, min_samples_split = 2), max_samples=1.0, n_estimators=5)
bgc3.fit(X_train_cls, y_train_cls)
print(bgc3.score(X_test_cls, y_test_cls))

0.87


## Адаптивный бустинг

### Документация 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

### Адаптивный бустинг без подбора гиперпараметров для него и базовых алгоритмов

In [20]:
from sklearn.ensemble import AdaBoostClassifier

In [21]:
modelClf7 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
modelClf7.fit(X_train_cls, y_train_cls)
print(modelClf7.score(X_test_cls, y_test_cls))

0.9583333333333334


#### Подбор гиперпараметров для бустинга

In [22]:
parameters = {
              'n_estimators':[50,60,70],
              'algorithm': ['SAMME', 'SAMME.R'],
             }
abc1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
grid5_ = GridSearchCV(abc1, parameters)

In [23]:
grid5_.fit(X_train_cls, y_train_cls)
print(grid5_.best_params_)
print(grid5_.best_estimator_)

{'algorithm': 'SAMME', 'n_estimators': 60}
AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=60)


In [24]:
abc2 = AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=70)
abc2.fit(X_train_cls, y_train_cls)
print(abc2.score(X_test_cls, y_test_cls))

0.9633333333333334


## Градиентный бустинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
modelClf8 = GradientBoostingClassifier()
modelClf8.fit(X_train_cls, y_train_cls)
print(modelClf8.score(X_test_cls, y_test_cls))

0.99


#### Подбор гиперпараметоров для градиентного бустинга

In [27]:
parameters = {
              'n_estimators':[100,150,200,300],
              'max_depth':[2,3,4,5],
              'criterion':['friedman_mse', 'squared_error'],
             }
gbc1 = GradientBoostingClassifier()
grid6_ = GridSearchCV(gbc1, parameters)

In [28]:
grid6_.fit(X_train_cls, y_train_cls)
print(grid6_.best_params_)
print(grid6_.best_estimator_)

{'criterion': 'friedman_mse', 'max_depth': 2, 'n_estimators': 150}
GradientBoostingClassifier(max_depth=2, n_estimators=150)


In [29]:
gbc2 = GradientBoostingClassifier(max_depth=2,criterion='friedman_mse', n_estimators = 100)
gbc2.fit(X_train_cls, y_train_cls)
print(gbc2.score(X_test_cls, y_test_cls))

0.9916666666666667


### Ансамбль методов для регрессии

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor

In [30]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [31]:
student_df = pd.read_csv('../data/student_math_preprocessed.csv')

student_df["useAlc"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(x, y,  test_size = 0.2, random_state=0)

In [32]:
estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
     ('lr', linear_model.LinearRegression())]
modelClf10 = StackingRegressor(estimators=estimators)

modelClf10.fit(X_train_reg, y_train_reg)
print(modelClf10.score(X_test_reg, y_test_reg))

0.8171296739083087


### Бэггинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor

In [33]:
from sklearn.ensemble import BaggingRegressor
modelClf12 = BaggingRegressor()
modelClf12.fit(X_train_reg, y_train_reg)
print(modelClf12.score(X_test_reg, y_test_reg))

0.8903309174672734


#### Подбор гиперпараметров для бэггинга

In [34]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgr2 = BaggingRegressor(base_estimator=LinearRegression())
grid8_ = GridSearchCV(bgr2, parameters)

In [35]:
grid8_.fit(X_train_reg, y_train_reg)
print(grid8_.best_params_)
print(grid8_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 10}
BaggingRegressor(base_estimator=LinearRegression())


In [36]:
modelClf14 = BaggingRegressor(base_estimator = DecisionTreeRegressor(),max_samples=1.0, n_estimators=20)
modelClf14.fit(X_train_reg, y_train_reg)
print(modelClf14.score(X_test_reg, y_test_reg))

0.8863981122534974


### Бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor

In [37]:
from sklearn.ensemble import AdaBoostRegressor
modelClf15 = AdaBoostRegressor()
modelClf15.fit(X_train_reg,y_train_reg)
print(modelClf15.score(X_test_reg, y_test_reg))

0.73958361305212


#### Градиентный бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
modelClf17 = GradientBoostingRegressor()
modelClf17.fit(X_train_reg, y_train_reg)
print(modelClf17.score(X_test_reg, y_test_reg))

0.8112821027502974


## Catboost

#### Открытый код 
https://github.com/catboost

In [39]:
!pip install catboost

In [51]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test_cls, y_test))

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import classification_report, confusion_matrix
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train_cls, y_train_cls)
print_classification_model_metrics(cbc, y_test_cls, cbc.predict(X_test_cls))

0:	learn: 0.4692953	total: 2.22ms	remaining: 2.22s
1:	learn: 0.3302720	total: 5.85ms	remaining: 2.92s
2:	learn: 0.2408309	total: 9.2ms	remaining: 3.06s
3:	learn: 0.1745150	total: 13.2ms	remaining: 3.29s
4:	learn: 0.1420410	total: 18.9ms	remaining: 3.75s
5:	learn: 0.1235028	total: 23.4ms	remaining: 3.87s
6:	learn: 0.1076271	total: 27.9ms	remaining: 3.95s
7:	learn: 0.0933984	total: 31.3ms	remaining: 3.88s
8:	learn: 0.0819392	total: 35ms	remaining: 3.85s
9:	learn: 0.0757649	total: 37.7ms	remaining: 3.73s
10:	learn: 0.0727758	total: 40.3ms	remaining: 3.62s
11:	learn: 0.0714588	total: 42.7ms	remaining: 3.52s
12:	learn: 0.0679745	total: 45.3ms	remaining: 3.44s
13:	learn: 0.0628354	total: 48.8ms	remaining: 3.44s
14:	learn: 0.0602450	total: 51.8ms	remaining: 3.4s
15:	learn: 0.0558751	total: 54.4ms	remaining: 3.35s
16:	learn: 0.0512265	total: 57ms	remaining: 3.3s
17:	learn: 0.0505344	total: 59.5ms	remaining: 3.25s
18:	learn: 0.0473527	total: 61.9ms	remaining: 3.2s
19:	learn: 0.0444205	total: 64

186:	learn: 0.0024819	total: 505ms	remaining: 2.19s
187:	learn: 0.0024595	total: 509ms	remaining: 2.2s
188:	learn: 0.0024579	total: 512ms	remaining: 2.19s
189:	learn: 0.0024399	total: 514ms	remaining: 2.19s
190:	learn: 0.0024398	total: 516ms	remaining: 2.19s
191:	learn: 0.0024389	total: 519ms	remaining: 2.18s
192:	learn: 0.0024387	total: 522ms	remaining: 2.18s
193:	learn: 0.0024159	total: 525ms	remaining: 2.18s
194:	learn: 0.0024149	total: 528ms	remaining: 2.18s
195:	learn: 0.0024002	total: 531ms	remaining: 2.18s
196:	learn: 0.0023972	total: 534ms	remaining: 2.18s
197:	learn: 0.0023959	total: 537ms	remaining: 2.17s
198:	learn: 0.0023508	total: 539ms	remaining: 2.17s
199:	learn: 0.0023262	total: 542ms	remaining: 2.17s
200:	learn: 0.0023262	total: 544ms	remaining: 2.16s
201:	learn: 0.0023262	total: 546ms	remaining: 2.16s
202:	learn: 0.0023262	total: 548ms	remaining: 2.15s
203:	learn: 0.0023262	total: 552ms	remaining: 2.15s
204:	learn: 0.0023261	total: 554ms	remaining: 2.15s
205:	learn: 0

400:	learn: 0.0013800	total: 1.15s	remaining: 1.72s
401:	learn: 0.0013800	total: 1.16s	remaining: 1.72s
402:	learn: 0.0013800	total: 1.16s	remaining: 1.72s
403:	learn: 0.0013800	total: 1.16s	remaining: 1.72s
404:	learn: 0.0013798	total: 1.17s	remaining: 1.71s
405:	learn: 0.0013797	total: 1.17s	remaining: 1.71s
406:	learn: 0.0013797	total: 1.17s	remaining: 1.71s
407:	learn: 0.0013712	total: 1.18s	remaining: 1.71s
408:	learn: 0.0013712	total: 1.18s	remaining: 1.7s
409:	learn: 0.0013711	total: 1.18s	remaining: 1.7s
410:	learn: 0.0013706	total: 1.19s	remaining: 1.7s
411:	learn: 0.0013704	total: 1.19s	remaining: 1.7s
412:	learn: 0.0013700	total: 1.19s	remaining: 1.69s
413:	learn: 0.0013699	total: 1.19s	remaining: 1.69s
414:	learn: 0.0013698	total: 1.2s	remaining: 1.69s
415:	learn: 0.0013693	total: 1.2s	remaining: 1.68s
416:	learn: 0.0013687	total: 1.2s	remaining: 1.68s
417:	learn: 0.0013687	total: 1.2s	remaining: 1.68s
418:	learn: 0.0013683	total: 1.21s	remaining: 1.67s
419:	learn: 0.001358

582:	learn: 0.0011563	total: 1.64s	remaining: 1.18s
583:	learn: 0.0011563	total: 1.65s	remaining: 1.17s
584:	learn: 0.0011563	total: 1.65s	remaining: 1.17s
585:	learn: 0.0011563	total: 1.65s	remaining: 1.17s
586:	learn: 0.0011561	total: 1.66s	remaining: 1.16s
587:	learn: 0.0011561	total: 1.66s	remaining: 1.16s
588:	learn: 0.0011561	total: 1.66s	remaining: 1.16s
589:	learn: 0.0011561	total: 1.67s	remaining: 1.16s
590:	learn: 0.0011561	total: 1.67s	remaining: 1.16s
591:	learn: 0.0011561	total: 1.68s	remaining: 1.16s
592:	learn: 0.0011560	total: 1.68s	remaining: 1.15s
593:	learn: 0.0011511	total: 1.69s	remaining: 1.15s
594:	learn: 0.0011510	total: 1.69s	remaining: 1.15s
595:	learn: 0.0011508	total: 1.69s	remaining: 1.15s
596:	learn: 0.0011508	total: 1.7s	remaining: 1.15s
597:	learn: 0.0011508	total: 1.7s	remaining: 1.14s
598:	learn: 0.0011508	total: 1.71s	remaining: 1.14s
599:	learn: 0.0011508	total: 1.71s	remaining: 1.14s
600:	learn: 0.0011508	total: 1.72s	remaining: 1.14s
601:	learn: 0.

754:	learn: 0.0009894	total: 2.3s	remaining: 745ms
755:	learn: 0.0009875	total: 2.3s	remaining: 742ms
756:	learn: 0.0009875	total: 2.3s	remaining: 739ms
757:	learn: 0.0009875	total: 2.31s	remaining: 737ms
758:	learn: 0.0009874	total: 2.31s	remaining: 733ms
759:	learn: 0.0009874	total: 2.31s	remaining: 730ms
760:	learn: 0.0009874	total: 2.31s	remaining: 727ms
761:	learn: 0.0009874	total: 2.32s	remaining: 724ms
762:	learn: 0.0009874	total: 2.32s	remaining: 721ms
763:	learn: 0.0009836	total: 2.33s	remaining: 718ms
764:	learn: 0.0009836	total: 2.33s	remaining: 715ms
765:	learn: 0.0009836	total: 2.33s	remaining: 713ms
766:	learn: 0.0009835	total: 2.33s	remaining: 709ms
767:	learn: 0.0009835	total: 2.34s	remaining: 706ms
768:	learn: 0.0009822	total: 2.34s	remaining: 703ms
769:	learn: 0.0009738	total: 2.34s	remaining: 700ms
770:	learn: 0.0009737	total: 2.34s	remaining: 696ms
771:	learn: 0.0009736	total: 2.35s	remaining: 694ms
772:	learn: 0.0009703	total: 2.35s	remaining: 690ms
773:	learn: 0.0

969:	learn: 0.0008784	total: 2.93s	remaining: 90.7ms
970:	learn: 0.0008784	total: 2.94s	remaining: 87.7ms
971:	learn: 0.0008784	total: 2.94s	remaining: 84.7ms
972:	learn: 0.0008784	total: 2.94s	remaining: 81.7ms
973:	learn: 0.0008784	total: 2.95s	remaining: 78.7ms
974:	learn: 0.0008784	total: 2.95s	remaining: 75.7ms
975:	learn: 0.0008783	total: 2.96s	remaining: 72.7ms
976:	learn: 0.0008783	total: 2.96s	remaining: 69.7ms
977:	learn: 0.0008716	total: 2.96s	remaining: 66.7ms
978:	learn: 0.0008716	total: 2.97s	remaining: 63.6ms
979:	learn: 0.0008715	total: 2.97s	remaining: 60.6ms
980:	learn: 0.0008709	total: 2.97s	remaining: 57.6ms
981:	learn: 0.0008709	total: 2.98s	remaining: 54.6ms
982:	learn: 0.0008708	total: 2.98s	remaining: 51.6ms
983:	learn: 0.0008708	total: 2.98s	remaining: 48.5ms
984:	learn: 0.0008708	total: 2.99s	remaining: 45.5ms
985:	learn: 0.0008708	total: 2.99s	remaining: 42.5ms
986:	learn: 0.0008601	total: 3s	remaining: 39.5ms
987:	learn: 0.0008600	total: 3s	remaining: 36.4ms

In [58]:
def print_regression_model_metrics(estimator, y_test, y_pred):
    print(estimator.score(x,y))
   

In [59]:
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train_reg, y_train_reg)
print_regression_model_metrics(cbr, y_test_reg, cbr.predict(X_test_reg))

0:	learn: 0.3173620	total: 2.58ms	remaining: 2.57s
1:	learn: 0.2854855	total: 5.15ms	remaining: 2.57s
2:	learn: 0.2623477	total: 8.73ms	remaining: 2.9s
3:	learn: 0.2370408	total: 11.6ms	remaining: 2.89s
4:	learn: 0.2184003	total: 14.5ms	remaining: 2.88s
5:	learn: 0.2017159	total: 17.4ms	remaining: 2.88s
6:	learn: 0.1857589	total: 20.5ms	remaining: 2.9s
7:	learn: 0.1713354	total: 23.4ms	remaining: 2.9s
8:	learn: 0.1608444	total: 27.3ms	remaining: 3s
9:	learn: 0.1531084	total: 30.1ms	remaining: 2.98s
10:	learn: 0.1432548	total: 33.1ms	remaining: 2.98s
11:	learn: 0.1363384	total: 35.8ms	remaining: 2.94s
12:	learn: 0.1315130	total: 38.9ms	remaining: 2.95s
13:	learn: 0.1275344	total: 42.1ms	remaining: 2.96s
14:	learn: 0.1218690	total: 44.3ms	remaining: 2.91s
15:	learn: 0.1178017	total: 47.9ms	remaining: 2.94s
16:	learn: 0.1145156	total: 50.2ms	remaining: 2.9s
17:	learn: 0.1116220	total: 52.3ms	remaining: 2.85s
18:	learn: 0.1098943	total: 54.7ms	remaining: 2.82s
19:	learn: 0.1086256	total: 5

222:	learn: 0.0443807	total: 497ms	remaining: 1.73s
223:	learn: 0.0443354	total: 500ms	remaining: 1.73s
224:	learn: 0.0442526	total: 503ms	remaining: 1.73s
225:	learn: 0.0442258	total: 507ms	remaining: 1.74s
226:	learn: 0.0440966	total: 510ms	remaining: 1.74s
227:	learn: 0.0438470	total: 513ms	remaining: 1.74s
228:	learn: 0.0435154	total: 518ms	remaining: 1.74s
229:	learn: 0.0434717	total: 522ms	remaining: 1.75s
230:	learn: 0.0431654	total: 526ms	remaining: 1.75s
231:	learn: 0.0430488	total: 530ms	remaining: 1.75s
232:	learn: 0.0429954	total: 533ms	remaining: 1.75s
233:	learn: 0.0429185	total: 535ms	remaining: 1.75s
234:	learn: 0.0428456	total: 537ms	remaining: 1.75s
235:	learn: 0.0426804	total: 539ms	remaining: 1.75s
236:	learn: 0.0426609	total: 541ms	remaining: 1.74s
237:	learn: 0.0425527	total: 543ms	remaining: 1.74s
238:	learn: 0.0423220	total: 546ms	remaining: 1.74s
239:	learn: 0.0421777	total: 548ms	remaining: 1.73s
240:	learn: 0.0421289	total: 550ms	remaining: 1.73s
241:	learn: 

407:	learn: 0.0253913	total: 978ms	remaining: 1.42s
408:	learn: 0.0253574	total: 981ms	remaining: 1.42s
409:	learn: 0.0253177	total: 984ms	remaining: 1.42s
410:	learn: 0.0252661	total: 987ms	remaining: 1.41s
411:	learn: 0.0252036	total: 990ms	remaining: 1.41s
412:	learn: 0.0251456	total: 993ms	remaining: 1.41s
413:	learn: 0.0251450	total: 995ms	remaining: 1.41s
414:	learn: 0.0251224	total: 998ms	remaining: 1.41s
415:	learn: 0.0251081	total: 1s	remaining: 1.41s
416:	learn: 0.0251008	total: 1s	remaining: 1.4s
417:	learn: 0.0250985	total: 1.01s	remaining: 1.4s
418:	learn: 0.0250341	total: 1.01s	remaining: 1.4s
419:	learn: 0.0249011	total: 1.01s	remaining: 1.4s
420:	learn: 0.0249007	total: 1.02s	remaining: 1.4s
421:	learn: 0.0248864	total: 1.02s	remaining: 1.4s
422:	learn: 0.0248200	total: 1.02s	remaining: 1.39s
423:	learn: 0.0247901	total: 1.02s	remaining: 1.39s
424:	learn: 0.0247373	total: 1.03s	remaining: 1.39s
425:	learn: 0.0247351	total: 1.03s	remaining: 1.38s
426:	learn: 0.0247285	to

575:	learn: 0.0173962	total: 1.46s	remaining: 1.07s
576:	learn: 0.0173703	total: 1.46s	remaining: 1.07s
577:	learn: 0.0173141	total: 1.46s	remaining: 1.07s
578:	learn: 0.0172729	total: 1.46s	remaining: 1.06s
579:	learn: 0.0172607	total: 1.47s	remaining: 1.06s
580:	learn: 0.0172163	total: 1.47s	remaining: 1.06s
581:	learn: 0.0171798	total: 1.47s	remaining: 1.06s
582:	learn: 0.0171386	total: 1.48s	remaining: 1.05s
583:	learn: 0.0170969	total: 1.48s	remaining: 1.05s
584:	learn: 0.0170589	total: 1.48s	remaining: 1.05s
585:	learn: 0.0170397	total: 1.48s	remaining: 1.05s
586:	learn: 0.0170125	total: 1.49s	remaining: 1.05s
587:	learn: 0.0169947	total: 1.49s	remaining: 1.04s
588:	learn: 0.0169275	total: 1.49s	remaining: 1.04s
589:	learn: 0.0168837	total: 1.49s	remaining: 1.04s
590:	learn: 0.0168349	total: 1.5s	remaining: 1.03s
591:	learn: 0.0167996	total: 1.5s	remaining: 1.03s
592:	learn: 0.0167832	total: 1.5s	remaining: 1.03s
593:	learn: 0.0167683	total: 1.5s	remaining: 1.03s
594:	learn: 0.01

753:	learn: 0.0123956	total: 1.94s	remaining: 632ms
754:	learn: 0.0123815	total: 1.94s	remaining: 630ms
755:	learn: 0.0123724	total: 1.94s	remaining: 627ms
756:	learn: 0.0123562	total: 1.95s	remaining: 624ms
757:	learn: 0.0123037	total: 1.95s	remaining: 622ms
758:	learn: 0.0122935	total: 1.95s	remaining: 619ms
759:	learn: 0.0122805	total: 1.95s	remaining: 617ms
760:	learn: 0.0122517	total: 1.96s	remaining: 614ms
761:	learn: 0.0122432	total: 1.96s	remaining: 612ms
762:	learn: 0.0122268	total: 1.96s	remaining: 609ms
763:	learn: 0.0121952	total: 1.96s	remaining: 607ms
764:	learn: 0.0121852	total: 1.97s	remaining: 604ms
765:	learn: 0.0121643	total: 1.97s	remaining: 602ms
766:	learn: 0.0121558	total: 1.97s	remaining: 599ms
767:	learn: 0.0121373	total: 1.97s	remaining: 596ms
768:	learn: 0.0121249	total: 1.98s	remaining: 594ms
769:	learn: 0.0120988	total: 1.98s	remaining: 591ms
770:	learn: 0.0120822	total: 1.98s	remaining: 588ms
771:	learn: 0.0120701	total: 1.98s	remaining: 586ms
772:	learn: 

936:	learn: 0.0093910	total: 2.44s	remaining: 164ms
937:	learn: 0.0093838	total: 2.44s	remaining: 161ms
938:	learn: 0.0093634	total: 2.44s	remaining: 159ms
939:	learn: 0.0093599	total: 2.45s	remaining: 156ms
940:	learn: 0.0093468	total: 2.45s	remaining: 154ms
941:	learn: 0.0093283	total: 2.45s	remaining: 151ms
942:	learn: 0.0093168	total: 2.46s	remaining: 149ms
943:	learn: 0.0093112	total: 2.46s	remaining: 146ms
944:	learn: 0.0092825	total: 2.47s	remaining: 144ms
945:	learn: 0.0092795	total: 2.47s	remaining: 141ms
946:	learn: 0.0092464	total: 2.47s	remaining: 138ms
947:	learn: 0.0092424	total: 2.48s	remaining: 136ms
948:	learn: 0.0092376	total: 2.48s	remaining: 133ms
949:	learn: 0.0092335	total: 2.48s	remaining: 131ms
950:	learn: 0.0092092	total: 2.48s	remaining: 128ms
951:	learn: 0.0092047	total: 2.49s	remaining: 125ms
952:	learn: 0.0091929	total: 2.49s	remaining: 123ms
953:	learn: 0.0091595	total: 2.49s	remaining: 120ms
954:	learn: 0.0091461	total: 2.49s	remaining: 118ms
955:	learn: 